In [1]:
import tensorflow as tf
print(tf.__version__)

physical_device = tf.config.experimental.list_physical_devices("GPU")
if len(physical_device) > 0:
    tf.config.experimental.set_memory_growth(physical_device[0], True)

2.2.0


In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.config.list_physical_devices('CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [5]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
x = tf.random.uniform([3, 3])

In [7]:
x.device

'/job:localhost/replica:0/task:0/device:GPU:0'

In [8]:
print('Is the Tensor on CPU #0:  ')
print(x.device.endswith('CPU:0'))
print('')
print('Is the Tensor on GPU #0:  ')
print(x.device.endswith('GPU:0'))

Is the Tensor on CPU #0:  
False

Is the Tensor on GPU #0:  
True


In [11]:
import time

def time_matadd(x):
    start = time.time()
    for loop in range(10):
        tf.add(x, x)
        
    result = time.time() - start
    print("Matrix addition (10 loops): {:0.2f}ms".format(1000*result))
    
def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)
        
    result = time.time() - start
    print('Matrix multiplication (10 loops): {:0.2f}ms'.format(1000*result))

In [16]:
print('On CPU:')
with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matadd(x)
    time_matmul(x)

On CPU:
Matrix addition (10 loops): 8.41ms
Matrix multiplication (10 loops): 64.39ms


In [17]:
if tf.config.experimental.list_physical_devices("GPU"):
    print("On GPU:")
    with tf.device("GPU:0"):
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matadd(x)
        time_matmul(x)

On GPU:
Matrix addition (10 loops): 0.19ms
Matrix multiplication (10 loops): 0.47ms


In [2]:
from tensorflow.keras.datasets import mnist
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255., x_test/255.

In [3]:
x_train, y_train = x_train[:1000], y_train[:1000]

In [4]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

def get_model():
    model = Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                      input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    return model

In [6]:
import time

with tf.device("CPU:0"):
    model = get_model()
    model.compile(optimizer=tf.keras.optimizers.RMSprop(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    start = time.time()
    model.fit(x_train[..., np.newaxis], y_train, epochs=5, verbose=0)
    result = time.time() - start

print("CPU training time: {:0.2f}ms".format(1000 * result))

CPU training time: 2798.24ms


In [10]:
with tf.device("GPU:0"):
    model = get_model()
    model.compile(optimizer=tf.keras.optimizers.RMSprop(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    start = time.time()
    model.fit(x_train[..., np.newaxis], y_train, epochs=5, verbose=0)
    result = time.time() - start

print("GPU training time: {:0.2f}ms".format(1000 * result))

GPU training time: 868.28ms
